In [1]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models.TorchAvitm import TorchAvitm
from gensim.models import Word2Vec

In [2]:
# Load dataset
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_5")

True

In [3]:
# Load model
model = TorchAvitm()

In [4]:
# Set model hyperparameters
model.hyperparameters['prior_variance'] = 0.2
model.hyperparameters['n_components'] = 5
model.hyperparameters['num_epochs'] = 1

In [5]:
model.hyperparameters


{'prior_variance': 0.2, 'n_components': 5, 'num_epochs': 1}

In [6]:
model.test_set(True)

In [7]:
model.train_model(dataset, model.hyperparameters)

Settings: 
               N Components: 5
               Topic Prior Mean: 0.0
               Topic Prior Variance: 0.2
               Model Type: prodLDA
               Hidden Sizes: (100, 100)
               Activation: softplus
               Dropout: 0.2
               Learn Priors: True
               Learning Rate: 0.002
               Momentum: 0.99
               Reduce On Plateau: False
               Save Dir: None
Epoch: [1/1]	Samples: [10840/10840]	Train Loss: 16921.15939806273	Time: 0:00:01.658081


{'topics': [['numerous',
   'strong',
   'usual',
   'cd',
   'motorcycle',
   'class',
   'junk',
   'hate',
   'russian',
   'offense'],
  ['identical',
   'schedule',
   'sure',
   'numerous',
   'routine',
   'class',
   'offense',
   'cd',
   'pack',
   'solution'],
  ['able',
   'absolute',
   'absolutely',
   'block',
   'gold',
   'permission',
   'technical',
   'claim',
   'instead',
   'virtual'],
  ['ability',
   'raid',
   'solar',
   'instead',
   'fully',
   'approach',
   'pin',
   'claim',
   'broad',
   'permission'],
  ['critical',
   'approach',
   'though',
   'newspaper',
   'money',
   'understand',
   'cheap',
   'question',
   'made',
   'chapter']],
 'topic-word-matrix': array([[ 0.06448709, -0.07448873, -0.05896251, ...,  0.09091718,
          0.13554999, -0.04824581],
        [ 0.04284371, -0.1182145 , -0.2066232 , ..., -0.02207693,
         -0.08595788, -0.02793388],
        [-0.12040485,  0.20028178,  0.14171745, ..., -0.14064965,
         -0.1648739 , -0.

In [8]:
model.inference()

ValueError: too many values to unpack (expected 6)

In [5]:
# Topic diversity
topic_diversity = Topic_diversity()

# KL_Uniform
#kl_uniform = KL_uniform()

In [6]:
# Define optimization parameters
opt_params = {}
opt_params["n_calls"] = 30
opt_params["minimizer"] = forest_minimize
opt_params["different_iteration"] = 3
opt_params["n_random_starts"] = 5
#opt_params["extra_metrics"] = [kl_uniform] # List of extra metrics
opt_params["n_jobs"] = mp.cpu_count() -1 # Enable multiprocessing
opt_params["verbose"] = True
opt_params["save_path"] = "results" #create folder if it doesn't exist 

In [7]:
# Create search space for optimization
search_space = {
    "num_epochs": Integer(low=1, high=50),
    #"eta": Real(low=0.01, high=5.0)
}

In [8]:
# Initialize optimizer
optimizer = Optimizer(
    model,
    dataset,
    topic_diversity,
    search_space,
    opt_params)

In [9]:
# Disable computing of topic document matrix to optimize performance
optimizer.topic_document_matrix = False
optimizer.topic_word_matrix = False

In [10]:
# Optimize
res = optimizer.optimize()

print(res.hyperparameters) # Best values for the hyperparameters
print(res.function_values) # Score of the optimized metric
print("Optimized metric: "+res.optimized_metric)

------------------------------------------
------------------------------------------
Bayesian optimization parameters:
-n_calls:  30 
-different_iteration:  3 
-n_random_starts:  5 
-minimizer:  forest_minimize 
-acq_func:  LCB 
-kernel:  1**2 * Matern(length_scale=1, nu=1.5)
------------------------------------------
Iteration No: 1 started. Evaluating function at random point.


KeyError: 'n_components'